# Imports

In [111]:
import geopandas
import pandas
import pandas as pd
import contextily as cx

In [29]:
gdf = pandas.read_csv("trips_04.csv",
                      on_bad_lines='skip',
                      parse_dates=['start_trip_dttm', 'day', 'end_trip_dttm'],
                      dtype={
                          "hash_ticket_uid": "string",
                          "start_latitude": "float32",
                          "start_longitude": "float32",
                          "start_entrance_id": "UInt32",
                          "start_entrance_nm": "string",
                          "start_station_id": "UInt32",
                          "start_station_nm": "string",
                          "start_line_id": "UInt32",
                          "start_line_nm": "string",
                          "start_stop_id": "UInt32",
                          "start_stop_nm": "string",
                          "vehicle_type": "string",
                          "end_station_id": "UInt32",
                          "end_station_nm": "string",
                          "end_line_id": "UInt32",
                          "end_line_nm": "string",
                          "end_stop_id": "UInt32",
                          "end_stop_nm": "string",
                          "end_latitude": "float32",
                          "end_longitude": "float32",
                          "route_num": "string"
                      }, nrows=1000000)

In [14]:
gdf.head(5)

,hash_ticket_uid,start_trip_dttm,start_latitude,start_longitude,start_entrance_id,start_entrance_nm,start_station_id,start_station_nm,start_line_id,start_line_nm,...,end_station_id,end_station_nm,end_line_id,end_line_nm,end_stop_id,end_stop_nm,end_longitude,end_longitude.1,trip_dur,subwindow_num
0,700702083475E41178E1F18CC7926A16,2022-04-04 14:56:40.0,55.705369,37.686200,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,1001331.0,МЦК Автозаводская,37.663325,37.663325,8.98,3.0
1,D4AB585E4AA790FBBAD4BF86CADB95B6,2022-04-04 16:45:47.0,55.779659,37.633563,195.0,Проспект Мира К,5011.0,Проспект Мира,5.0,Кольцевая,...,2005.0,Водный стадион,2.0,Замоскворецкая,NaN,NaN,37.486795,37.486795,22.15,2.0
2,C1E6D455DED5D7DBD7A63E437FC749AA,2022-04-04 10:44:35.0,55.855686,37.354120,2101.0,Пятницкое шоссе (Первый),3016.0,Пятницкое шоссе,3.0,Арбатско-Покровская,...,9009.0,Нагатинская,9.0,Серпуховско-Тимирязевская,NaN,NaN,37.622125,37.622125,56.75,1.0
3,33704A9BE7B8183CAF97BB01B1FEFDDD,2022-04-04 13:55:02.0,55.724571,37.818664,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,15936.0,"Метро ""Выхино"" (выс.)",37.817305,37.817305,2.02,1.0
4,ADA858BB46FAA6C46187FD7F8B7B555D,2022-04-04 17:27:46.0,55.707654,37.586216,266.0,Ленинский проспект (Северный),6010.0,Ленинский проспект,6.0,Калужско-Рижская,...,6020.0,Тёплый Стан,6.0,Калужско-Рижская,NaN,NaN,37.507044,37.507044,16.63,2.0


## Станции

In [27]:
stations = gdf[['start_latitude', 'start_longitude', 'start_station_id', 'start_station_nm']].drop_duplicates(
    subset="start_station_id").copy()
stations = stations.dropna()
stations

NameError: name 'gdf' is not defined

In [23]:
count = gdf.start_station_id.value_counts()
stations['start_stations_count'] = count.values
stations

,start_latitude,start_longitude,start_station_id,start_station_nm,start_count,geometry,start_stations_count
1,55.779659,37.633563,5011.0,Проспект Мира,34002,POINT (37.63356 55.77966),34002
2,55.855686,37.354120,3016.0,Пятницкое шоссе,26298,POINT (37.35412 55.85569),26298
4,55.707654,37.586216,6010.0,Ленинский проспект,24440,POINT (37.58622 55.70765),24440
5,55.740683,37.625617,8007.0,Третьяковская,22604,POINT (37.62562 55.74068),22604
9,55.815710,37.737522,14006.0,Бульвар Рокоссовского,22092,POINT (37.73752 55.81571),22092
...,...,...,...,...,...,...,...
354080,55.635946,37.066632,1001036.0,Дачное,7,POINT (37.06663 55.63595),7
456740,55.588704,36.744167,1001046.0,Портновская,5,POINT (36.74417 55.58870),5
1467163,55.596190,36.786336,1001045.0,Часцовская,3,POINT (36.78634 55.59619),3
1538979,55.678608,36.913609,1001041.0,Скоротово,3,POINT (36.91361 55.67861),3


In [24]:
st = geopandas.GeoDataFrame(
    stations, geometry=geopandas.points_from_xy(stations.start_longitude, stations.start_latitude))
st = st.dropna(subset=['start_longitude']).dropna(subset=['start_latitude'])

st


,start_latitude,start_longitude,start_station_id,start_station_nm,start_count,geometry,start_stations_count
1,55.779659,37.633563,5011.0,Проспект Мира,34002,POINT (37.63356 55.77966),34002
2,55.855686,37.354120,3016.0,Пятницкое шоссе,26298,POINT (37.35412 55.85569),26298
4,55.707654,37.586216,6010.0,Ленинский проспект,24440,POINT (37.58622 55.70765),24440
5,55.740683,37.625617,8007.0,Третьяковская,22604,POINT (37.62562 55.74068),22604
9,55.815710,37.737522,14006.0,Бульвар Рокоссовского,22092,POINT (37.73752 55.81571),22092
...,...,...,...,...,...,...,...
354080,55.635946,37.066632,1001036.0,Дачное,7,POINT (37.06663 55.63595),7
456740,55.588704,36.744167,1001046.0,Портновская,5,POINT (36.74417 55.58870),5
1467163,55.596190,36.786336,1001045.0,Часцовская,3,POINT (36.78634 55.59619),3
1538979,55.678608,36.913609,1001041.0,Скоротово,3,POINT (36.91361 55.67861),3


In [27]:
st.explore(legend=True, column='start_stations_count')

In [25]:
from ipyleaflet import Map, GeoJSON, GeoData
from ipyleaflet import Map, basemaps

center = [38.128, 2.588]
zoom = 5

m = Map(basemap=basemaps.OpenStreetMap.Mapnik, center=center, zoom=zoom)
geo_data = GeoData(geo_dataframe=st,
                   style={'color': 'black', 'radius': 8, 'fillColor': '#3366cc', 'opacity': 0.5, 'weight': 1,
                          'dashArray': '2', 'fillOpacity': 0.6},
                   hover_style={'fillColor': 'red', 'fillOpacity': 0.2},
                   point_style={'radius': 5, 'color': 'red', 'fillOpacity': 0.8, 'fillColor': 'blue', 'weight': 3},
                   name='Release')

m.add_layer(geo_data)
m

Map(center=[38.128, 2.588], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title', 'zoom_…

## Остановки

In [26]:
stops = gdf[['start_latitude', 'start_longitude', 'start_stop_id', 'start_stop_nm']].drop_duplicates(
    subset="start_stop_id").copy()
stops = stops.dropna()
stops

,start_latitude,start_longitude,start_stop_id,start_stop_nm
0,55.705369,37.686200,8474.0,"""Метро """"Кожуховская"""""""
3,55.724571,37.818664,4392.0,"Снайперская ул., 3"
6,55.827171,37.763657,3948.0,Детские ясли-сад
7,55.876019,37.557451,1352.0,"Дубнинская ул., 28"
8,55.712200,37.757068,6070.0,"Окская ул., 18"
...,...,...,...,...
3737915,55.719019,37.902305,14608.0,Красковская ул.
3739621,55.814202,37.733861,7123.0,"""Метро """"Бульв. Рокоссовского"""""""
3752836,55.449199,36.893206,17360.0,Поворот на Дубки
3761667,55.757523,37.432260,1937.0,Стадион


In [27]:
start_count = gdf.start_stop_id.value_counts()
stops['start_stops_count'] = start_count.values
stops

,start_latitude,start_longitude,start_stop_id,start_stop_nm,start_stops_count
0,55.705369,37.686200,8474.0,"""Метро """"Кожуховская""""""",2272
3,55.724571,37.818664,4392.0,"Снайперская ул., 3",2254
6,55.827171,37.763657,3948.0,Детские ясли-сад,2060
7,55.876019,37.557451,1352.0,"Дубнинская ул., 28",1811
8,55.712200,37.757068,6070.0,"Окская ул., 18",1640
...,...,...,...,...,...
3737915,55.719019,37.902305,14608.0,Красковская ул.,1
3739621,55.814202,37.733861,7123.0,"""Метро """"Бульв. Рокоссовского""""""",1
3752836,55.449199,36.893206,17360.0,Поворот на Дубки,1
3761667,55.757523,37.432260,1937.0,Стадион,1


In [13]:
from ipyleaflet import Map, GeoJSON, GeoData
from ipyleaflet import Map, basemaps

center = [38.128, 2.588]
zoom = 5

m = Map(basemap=basemaps.OpenStreetMap.Mapnik, center=center, zoom=zoom)
geo_data = GeoData(geo_dataframe=st,
                   style={'color': 'black', 'radius': 8, 'fillColor': '#3366cc', 'opacity': 0.5, 'weight': 1,
                          'dashArray': '2', 'fillOpacity': 0.6},
                   hover_style={'fillColor': 'red', 'fillOpacity': 0.2},
                   point_style={'radius': 5, 'color': 'red', 'fillOpacity': 0.8, 'fillColor': 'blue', 'weight': 3},
                   name='Release')

m.add_layer(geo_data)
m

Map(center=[38.128, 2.588], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title', 'zoom_…

## Популярные пересадки

In [5]:
import json
import pickle

with open("test.bytes", "rb") as f:
    obj = pickle.load(f)

print(obj[51])

[day                               2022-04-04 00:00:00
hash_ticket_uid      0004F69F29FDFD04526200DEF2DE2192
start_trip_dttm             2022-04-04 11:14:47+03:00
start_latitude                              55.697159
start_longitude                              37.46447
start_entrance_id                                2459
start_entrance_nm                Аминьевская (Первый)
start_station_id                                11018
start_station_nm                          Аминьевская
start_line_id                                      11
start_line_nm                       Большая кольцевая
start_stop_id                                    <NA>
start_stop_nm                                    <NA>
route_num                                        <NA>
vehicle_type                                     <NA>
end_trip_dttm               2022-04-04 11:36:02+03:00
end_station_id                                   7013
end_station_nm                           Полежаевская
end_line_id                

In [67]:
import numpy as np
import math
import pandas
import datetime

changes = []
changes: list[dict]
for completed_trip in obj:
    if len(completed_trip) >= 1:
        for i, change_obj in enumerate(zip(completed_trip, completed_trip[1:])):
            change_n, change_n1 = change_obj
            duration = pandas.to_datetime(change_n1['start_trip_dttm']) - pandas.to_datetime(change_n['end_trip_dttm'])
            if abs(change_n1['start_latitude'] - change_n['end_latitude']) > 0.03 or duration < datetime.timedelta(
                seconds=0) or abs(change_n1['start_longitude'] - change_n['end_longitude']) > 0.03:
                break
            end_id = change_n["end_stop_id"] if not pandas.isna(change_n["end_stop_id"]) else change_n["end_station_id"]
            start_id = change_n1["start_stop_id"] if not pandas.isna(change_n1["start_stop_id"]) else change_n1[
                "start_station_id"]
            end_name = change_n["end_stop_nm"] if not pandas.isna(change_n["end_stop_nm"]) else change_n[
                "end_station_nm"]
            start_name = change_n1["start_stop_nm"] if not pandas.isna(change_n1["start_stop_nm"]) else change_n1[
                "start_station_nm"]
            print(f"{end_name} ({end_id}) -> {start_name} ({start_id})")
            changes.append({"name": f"{end_name} ({end_id}) -> {start_name} ({start_id})",
                            "start_id": start_id, "end_id": end_id,
                            "duration": duration,
                            "start_latitude": change_n1["start_latitude"],
                            "start_longitude": change_n1["start_longitude"],
                            "end_latitude": change_n["end_latitude"],
                            "end_longitude": change_n["end_longitude"],
                            'hash_ticket_uid': change_n['hash_ticket_uid']
                            })

Петровский парк (11002) -> Метро \Динамо\"" (14767)
\Метро \"\"Давыдково\"\"\"" (2120) -> Давыдково (11019)
Ул. Куусинена, 13 (1727) -> Ул. Куусинена, 13 (1727)
\Метро \"\"Выхино\"\"\"" (15936) -> Выхино (7004)
Ул. Короленко - Социальный Университет (3832) -> Малая Остроумовская улица (6365)
Партизанская (3013) -> Метро \Партизанская\"" (7889)
Площадь Гагарина (14024) -> Ленинский проспект (6010)
Бульвар Рокоссовского (1002) -> \Метро \"\"Бульв. Рокоссовского\"\"\"" (8604)
Пражская (9015) -> \Метро \"\"Пражская\"\"\"" (2741)
Нижегородская (14020) -> Нижегородская (15005)
Щукинская (7023) -> Щукинская (1002055)
\Метро \"\"Октябрьская\"\" (радиальная)\"" (2296) -> Октябрьская (6014)
\Метро \"\"Профсоюзная\"\"\"" (16127) -> Профсоюзная (6016)
\Метро \"\"Царицыно\"\"\"" (6976) -> Царицыно (2023)
Электрозаводская (3023) -> Электрозаводская (3023)
\Метро \"\"Фрунзенская\"\"\"" (6309) -> Фрунзенская (1019)
Беляево (6004) -> \Метро \"\"Беляево\"\"\"" (11310)
Селигерская (10018) -> \Метро \"\"С

In [83]:
change_df = pandas.DataFrame(changes)
durations = map(lambda x: str(x), change_df['duration'])
change_df['duration'] = list(durations)
counter = change_df[["name"]].value_counts()
change_df = change_df.drop_duplicates(subset="name")
change_df['change_counter'] = counter.values
change_df

,name,start_id,end_id,duration,start_latitude,start_longitude,end_latitude,end_longitude,hash_ticket_uid,change_counter
0,"Петровский парк (11002) -> Метро \Динамо\"""" (1...",14767,11002,0 days 00:04:06,55.791786,37.555794,55.791824,37.557667,0038CEE1EC6CE10AAE23ED614256FA3A,43
1,"\Метро \""\""Давыдково\""\""\"""" (2120) -> Давыдков...",11019,2120,0 days 00:04:18,55.714642,37.451939,55.715050,37.453011,0077A0AFF6C723807F819AA2ABACFE1E,25
2,"Ул. Куусинена, 13 (1727) -> Ул. Куусинена, 13 ...",1727,1727,0 days 00:16:41,55.784218,37.514164,55.784218,37.514164,00805D1767F76BBB65B3BCEF38B58BE2,24
3,"\Метро \""\""Выхино\""\""\"""" (15936) -> Выхино (7004)",7004,15936,0 days 00:06:29,55.715889,37.818111,55.717346,37.816673,00A19F8C6D3C8DCD5A44EB5A7E0A21F2,24
4,Ул. Короленко - Социальный Университет (3832) ...,6365,3832,0 days 00:03:50,55.793194,37.690769,55.793812,37.693203,00B81D4B052B654DD78F533B6197912F,22
...,...,...,...,...,...,...,...,...,...,...
2636,Чистые пруды (1021) -> Б. Харитоньевский пер. ...,7581,1021,0 days 00:11:45,55.763149,37.642902,55.764900,37.638390,FF1ABD06B081847937CF1F2C6B513B5E,1
2638,Ул. М. Дмитровка (3371) -> Пушкинская (7015),7015,3371,0 days 00:04:46,55.765144,37.605049,55.766785,37.606400,FF2B046B97738FD879641E9F9E06FB5B,1
2640,"Митино (3009) -> \Метро \""\""Митино\""\""\"""" (15659)",15659,3009,0 days 00:06:15,55.847103,37.361774,55.845982,37.361320,FF56E21572FA663E93D0A3CEA6862FF3,1
2642,Ул. Серегина (386) -> Ул. Серегина (386),386,386,0 days 00:08:36,55.794853,37.546028,55.794853,37.546028,FF9C8E461612969DE29C2CF76A89DB4B,1


In [90]:
change_df = change_df[change_df.change_counter > 0]
geometry = [LineString((sll_ell[:2], sll_ell[2:])) for sll_ell in
            zip(change_df.start_longitude, change_df.start_latitude, change_df.end_longitude, change_df.end_latitude)]
# m = gpd.GeoDataFrame(geometry=[cao_poly], crs="epsg:4326").explore()
changes_gdf = geopandas.GeoDataFrame(change_df, geometry=geometry, crs="epsg:4386")
m = changes_gdf.explore(column='change_counter', cmap='coolwarm')
m

In [25]:
from pathlib import Path
import pandas as pd
import geopandas as gpd
import shapely
import folium

# downloaded data
geo_stations = geopandas.GeoDataFrame(
    stations, geometry=geopandas.points_from_xy(stations.start_longitude, stations.start_latitude), crs="epsg:4386")
geo_stops = geopandas.GeoDataFrame(
    stops, geometry=geopandas.points_from_xy(stops.start_longitude, stops.start_latitude), crs="epsg:4386")

# plot cao city limits
with open('poly.txt', 'r') as poly:
    cao_poly = shapely.wkt.loads(
        poly.read()
    )
# plot belgrade city limits
m = gpd.GeoDataFrame(geometry=[cao_poly], crs="epsg:4326").explore()
# plot the points, just for demo purposes plot outcomes as different colors
m = geo_stations.explore(m=m, column="start_stations_count", name="stations", cmap='spring')
m = geo_stops.explore(m=m, column="start_stops_count", name="stops", cmap='winter')
# add layer control so layers can be switched on / off
folium.LayerControl().add_to(m)

m

NameError: name 'stations' is not defined

## Смотрим пересадочные поездки с большим потоком

In [145]:
cleared_change_trips = list()
for trips in obj:
    if len(trips) > 1:
        name = f'{" → ".join([trip["start_station_nm"] if not pandas.isna(trip["start_station_nm"]) else trip["start_stop_nm"] for trip in trips])}'
        cleared_change_trips.append(
            {'name': f"{name} → {trips[-1]['end_station_nm'] if not pandas.isna(trips[-1]['end_station_nm']) else trips[-1]['end_stop_nm']}",
             'len': len(trips),
             'start_id': trips[0]["start_station_id"] if not pandas.isna(trips[0]["start_station_id"]) else trips[0][
                 "start_stop_id"],
             'stop_id': trips[-1]["start_station_id"] if not pandas.isna(trips[-1]["start_station_id"]) else trips[-1][
                 "start_stop_id"],
             'start_latitude': trips[0].start_latitude,
             'start_longitude': trips[0].start_longitude,
             'end_latitude': trips[-1].end_latitude,
             'end_longitude': trips[-1].end_longitude,
             })
        if '→' not in cleared_change_trips[-1]['name']:
            print(cleared_change_trips[-1])
cleared_change_trips

[{'name': 'Сокол → Метро \\Динамо\\"" → Петровско-Разумовская аллея',
  'len': 2,
  'start_id': 2018,
  'stop_id': 14767,
  'start_latitude': 55.80480194091797,
  'start_longitude': 37.51553726196289,
  'end_latitude': 55.795235,
  'end_longitude': 37.559795},
 {'name': 'Ул. Генерала Дорохова → Давыдково → Волоколамская',
  'len': 2,
  'start_id': 2117,
  'stop_id': 11019,
  'start_latitude': 55.71056365966797,
  'start_longitude': 37.4452018737793,
  'end_latitude': 55.83554,
  'end_longitude': 37.382347},
 {'name': '\\Метро \\"\\"Полежаевская\\"\\"\\"" → Ул. Куусинена, 13 → Песчаная пл.',
  'len': 2,
  'start_id': 1063,
  'stop_id': 1727,
  'start_latitude': 55.77775192260742,
  'start_longitude': 37.51759338378906,
  'end_latitude': 55.793453,
  'end_longitude': 37.51091},
 {'name': '\\Метро \\"\\"Выхино\\"\\"\\"" → Выхино → Перово',
  'len': 2,
  'start_id': 4430,
  'stop_id': 7004,
  'start_latitude': 55.7172737121582,
  'start_longitude': 37.82025909423828,
  'end_latitude': 55.7

In [166]:
change_trips = pd.DataFrame(cleared_change_trips)
counter = change_trips[["name"]].value_counts()
change_trips = change_trips.drop_duplicates(subset="name")
change_trips
change_trips['change_counter'] = counter.values
change_trips

,name,len,start_id,stop_id,start_latitude,start_longitude,end_latitude,end_longitude,change_counter
0,"Сокол → Метро \Динамо\"""" → Петровско-Разумовск...",2,2018,14767,55.804802,37.515537,55.795235,37.559795,3
1,Ул. Генерала Дорохова → Давыдково → Волоколамская,2,2117,11019,55.710564,37.445202,55.835541,37.382347,3
2,"\Метро \""\""Полежаевская\""\""\"""" → Ул. Куусинена...",2,1063,1727,55.777752,37.517593,55.793453,37.510910,3
3,"\Метро \""\""Выхино\""\""\"""" → Выхино → Перово",2,4430,7004,55.717274,37.820259,55.751118,37.785522,2
4,Зельев пер. → Малая Остроумовская улица → Лесн...,2,3901,6365,55.797493,37.722775,55.781570,37.670147,2
...,...,...,...,...,...,...,...,...,...
2784,9-й мкр. Марьинского Парка → Братиславская → В...,2,10397,10002,55.661865,37.765743,55.715889,37.818111,1
2785,"\Отель \""\""Советский\""\"" - Театр \""\""Ромэн\""\""...",2,1017,386,55.785236,37.567726,55.797798,37.539299,1
2786,"Шелепиха → \Метро \""\""Алтуфьево\""\""\"""" → Забол...",2,11005,1002648,55.756874,37.523819,55.926052,37.547050,1
2787,Ул. Гаврикова → Бауманская → Давыдково,2,8165,3002,55.784592,37.664967,55.714642,37.451939,1


In [167]:
geometry = [LineString((sll_ell[:2], sll_ell[2:])) for sll_ell in
            zip(change_trips.start_longitude, change_trips.start_latitude, change_trips.end_longitude,
                change_trips.end_latitude)]
change_trips_gdf = geopandas.GeoDataFrame(change_trips, geometry=geometry, crs="epsg:4386")
m = change_trips_gdf.explore()
m